<a href="https://colab.research.google.com/github/nik8839/deep_learning_project/blob/trial_1/vit_increased_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
#from tensorflow.keras.applications import Vit

# Rest of the code

from PIL import Image



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:



# Load your image data
classes = []
data = []
Y = []
image_folder_path = '/content/drive/MyDrive/dataset'  # Change this to the path where your folders are extracted
folders = os.listdir(image_folder_path)
count = 0
for folder in folders:
    classes.append(folder)
    folder_path = os.path.join(image_folder_path, folder)
    if os.path.isdir(folder_path):
        print(f"Reading images from {folder}")
        images = os.listdir(folder_path)
        for image_name in images:
            Y.append(count)
            image_path = os.path.join(folder_path, image_name)
            image = Image.open(image_path)
            image = image.resize((112,112), resample=Image.LANCZOS).convert('RGB')
            image_array = np.asarray(image)
            data.append(image_array)

        count += 1



Reading images from lightning
Reading images from rime
Reading images from glaze
Reading images from rainbow
Reading images from hail
Reading images from frost
Reading images from sandstorm
Reading images from rain
Reading images from snow
Reading images from fogsmog
Reading images from dew


In [31]:
# Convert lists to numpy arrays
data = np.array(data)
Y = np.array(Y)


In [32]:

# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(data, Y, test_size=0.2, random_state=42)


In [14]:
pip install vit_keras


In [15]:
pip install tensorflow_addons

In [33]:
import tensorflow as tf
from tensorflow.keras import layers
from vit_keras import vit, utils

# Assuming you have already loaded and preprocessed your data (X_train, Y_train, X_val, Y_val)

# Define the Vision Transformer model
def create_vit_model(image_size, num_classes):
    model = vit.vit_b16(
        image_size=image_size,
        activation='softmax',
        pretrained=True,  # Set to True if you want to use pre-trained weights
        include_top=True,
        classes=num_classes,
    )
    return model

# Create the ViT model
input_shape = (112,112)
num_classes = 11  # Adjust based on your specific classification task
vit_model = create_vit_model(input_shape, num_classes)

# Compile the model
vit_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
vit_model.summary()

# Train the model
vit_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=20, batch_size=25)

# Evaluate the model
eval_result = vit_model.evaluate(X_val, Y_val)
print("Validation Loss:", eval_result[0])
print("Validation Accuracy:", eval_result[1])


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 7, 7
  warnings.warn(


Model: "vit-b16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 768)         590592    
                                                                 
 reshape_7 (Reshape)         (None, 49, 768)           0         
                                                                 
 class_token (ClassToken)    (None, 50, 768)           768       
                                                                 
 Transformer/posembed_input  (None, 50, 768)           38400     
  (AddPositionEmbs)                                              
                                                                 
 Transformer/encoderblock_0  ((None, 50, 768),         7087872   
  (TransformerBlock)          (None, 12, None, None)       